In [ ]:
import os
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np  
#from skimage.metrics import structural_similarity
#from sklearn.cluster import KMeans
import random
#import openslide
import copy

#import matplotlib.image

In [ ]:
# Code for NRB

In [ ]:
def cossim(Q, K):
    K_norm = np.linalg.norm(K, axis=1, keepdims=True)
    Q_norm = np.linalg.norm(Q, axis=1, keepdims=True)
    QK_norm = Q_norm @ K_norm.T
    QK = Q @ K.T

    return QK / QK_norm

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
def computeRepresentativeVectors(WSI, tumorLabels, locPts, threshold):    
    NRB = np.zeros([1, 1024])
    #tumorLabels = []
    #locPts = []
    
    bag = np.load(WSI)
    
    wsiName = WSI.split("/")[-1].split(".")[0]
    
    #WSILabels = np.load(tumorLabelsPath.format(wsiName))
    #locsPoints = np.load("./CM16/Ziyu/locs/{}.npy".format(wsiName))
    
    while(len(bag) != 0):
        randomNo = random.randint(0, len(bag) - 1)
        tumorLabels.append(WSILabels[randomNo])
        locPts.append(locsPoints[randomNo])
        
        randomFE = bag[randomNo, :].reshape(1, -1)

        bag = np.delete(bag, randomNo, 0)
        WSILabels = np.delete(WSILabels, randomNo, 0)
        
        NRB = np.append(NRB, randomFE, axis = 0)

        if (len(bag) > 0):
            res = cossim(randomFE, bag)
            maxSim = np.max(res)

            if (maxSim > threshold):
                bag = np.delete(bag, np.argwhere(res > threshold)[:, 1], 0)
                WSILabels = np.delete(WSILabels, np.argwhere(res > threshold)[:, 1], 0)
                locsPoints = np.delete(locsPoints, np.argwhere(res > threshold)[:, 1], 0)
                
    return NRB[1:, :], tumorLabels, locPts

In [ ]:
### Creation of NRB using 0.95 tumor threshold

In [ ]:
className = "normal"
baseDir = "./CM16/Ziyu/feats/train/{}/".format(className)

for threshold in [0.95]:
    tumorLabelsPath = "./CM16/Ziyu/locs/labels/{}labels.npy"

    labelsArr = {}

    for WSI in os.listdir(baseDir):
        print(WSI)
        NRB, tumorLabels, locs = computeRepresentativeVectors(baseDir + WSI, tumorLabelsPath, threshold)
        
        np.save("./CM16/Ziyu/feats_2_{}/train/{}/{}".format(threshold, className, WSI), NRB)
        if (className != "normal"):
            np.save("./CM16/Ziyu/feats_2_{}/train/{}/{}labels.npy".format(threshold, className, WSI.split(".")[0]), tumorLabels)
        np.save("./CM16/Ziyu/feats_2_{}/locs/{}".format(threshold, WSI), locs)
        
        labelsArr[WSI] = sum(tumorLabels)